In [ ]:
'''
YOUR TASKS:

1. SPECIFY THE URL OR PATH TO YOUR DATA, REMEMBER - YOU FIRST HAVE TO DOWNLOAD IT.
2. CLICK RUN ALL OR RESTART AND RUN ALL BUTTON TO RUN THE CODE. THIS CODE RUNS IN A CHRONOLOGICAL MANNER - THE TOP BLOCKS MUST PRECEED THE BOTTOM ONES,
                                                                                                            i.e. don't run stuff from the bottom first, otherwise the code won't work properly.
3. NEAR THE END OF THE CODE, YOU WILL SEE YOUR EPOCH METRICS, AS I STATED IN CLASS, YOU WANT THE ACCURACY TO APPROACH 100% =1.0 AND THE LOSS TO APPROACH 0
4. IN CASE IF YOUR MODEL IS STRUGGLING TO REACH THESE RESULTS AND IS INSTEAD STUCK AT A LOW ACCURACY, NAVIGATE TO THE BLOCK OF CODE WITH HYPERPARAMETERS AND CHANGE THESE NUMBERS,
THEN, TRY RUNNING THE CODE AGAIN AND SEE HOW YOUR MODEL IS PERFORMING; REPEAT UNTIL YOU REACH A PLAUSIBLE RESULT (ONE THAT IS BETTER THAN A COIN TOSS IN TERMS OF PREDICTION ACCURACY IN THE WORST CASE)
5. SAVE YOUR RESULTS => THE CODE INCLUDES A BLOCK WHICH WILL SAVE YOUR MODEL TO A .h5 FILE,
 - KEEP THE OUTPUTS OF THIS NOTEBOOK AS WELL, YOU WILL WANT TO DEMONSTRATE HOW THE TRAINING PROCESS WENT.
 - BE MINDFUL OF THE TIME YOU HAVE IF YOU CHOOSE TO USE GOOGLE COLLABORATORY INSTEAD OF YOUR OWN MACHINE AS THEIR VMs ONLY OFFER YOU A COUPLE OF HOURS OF USAGE.
'''

In [32]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import time

In [ ]:
# Function to read the data
def read_data(filename):
    data = pd.read_csv(filename)

    # Extract features
    features = data.iloc[:,1 :-1].values

    # Extract and encode labels
    label_encoder = LabelEncoder()
    data.iloc[:, -1] = label_encoder.fit_transform(data.iloc[:, -1])
    labels = data.iloc[:, -1].values

    return features, labels
# Read the data
data_url = "https://raw.githubusercontent.com/camilleb2700/ENGIN-442/main/user_4.csv" # enter your data URL or PATH here in enclosed in " " IF YOUR NOTEBOOK IS IN THE SAME FOLDER AS data.csv, which is your data you downloaded, then this will run as it is.
                    # otherwise, you have to specify a different path or url
x, y = read_data(data_url)

In [34]:
df = pd.read_csv(data_url) ## THIS AND THE BLOCK OF CODE IMMEDIATELY AFTER THIS ARE OPTIONAL, THIS IS INTENDED TO HELP YOU VISUALIZE YOUR DATASET

In [35]:
df # AS STATED ABOVE, YOU MAY REMOVE THIS AND THE ABOVE BLOCKS OF CODE IF YOU LIKE, THIS WILL HAVE NO

,Time (Seconds),x-rotation,y-rotation,z-rotation,x-position,y-position,z-position,label
0,0.000000,71.503227,8.469132,15.086559,1.031810,1.135602,-0.420733,Sit_VidCall_Phone
1,0.008333,71.465935,8.534703,15.115500,1.031648,1.135565,-0.420697,Sit_VidCall_Phone
2,0.016667,71.402290,8.547145,15.212766,1.031494,1.135499,-0.420653,Sit_VidCall_Phone
3,0.025000,71.345825,8.632352,15.297141,1.031372,1.135447,-0.420621,Sit_VidCall_Phone
4,0.033333,71.296577,8.698273,15.370334,1.031248,1.135412,-0.420604,Sit_VidCall_Phone
...,...,...,...,...,...,...,...,...
80469,30.300000,25.907475,-53.189522,126.003304,-0.335001,1.214560,-0.059039,Walk_WebBrowse_Tablet
80470,30.308333,26.524557,-53.891808,126.768288,-0.337853,1.216213,-0.051177,Walk_WebBrowse_Tablet
80471,30.316667,27.122971,-54.681313,127.514549,-0.340846,1.217775,-0.043328,Walk_WebBrowse_Tablet
80472,30.325000,27.821383,-55.441963,128.452454,-0.344030,1.219314,-0.035561,Walk_WebBrowse_Tablet


In [36]:
x.shape ### IN THE FOLLOWING PARTS OF THE CODE, MAKE SURE THAT EVERYTING IS LOADED CORRECTLY, YOU SHOULD HAVE A DATASET WITH 22 CLASSES, (n-rows, 6 columns, later another dimension will be added here)

(80474, 6)

In [37]:
y.shape

(80474,)

In [38]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21])

In [39]:
x[0]

array([71.503227,  8.469132, 15.086559,  1.03181 ,  1.135602, -0.420733])

In [40]:
y[0] # these were sample rows from the data for your to inspect, you should see a 6 by 1 vector above and a single value here

0

In [41]:
# Split the data into training, validation, and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [42]:
# Reshape the data
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [43]:
# Determine the input shape and number of classes
input_shape = (x_train.shape[1], 1)
num_classes = len(np.unique(y))

In [44]:
num_classes # must have 22

22

In [45]:
input_shape

(6, 1)

In [46]:
x_train.shape

(57940, 6, 1)

In [47]:
y_train.shape

(57940,)

In [48]:
x_train[0]

array([[-76.838287],
       [-16.723667],
       [-74.727928],
       [  0.632294],
       [  1.719963],
       [  0.247437]])

In [49]:
y_train[0]

13

In [50]:
# Transformer Encoder function with dropout and L2 regularization
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1, l2_reg=0.01):

    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed-forward network with dropout and L2 regularization
    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(res)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    '''
    YOU MAY WANT TO ADD OR REMOVE THESE TWO LINES HERE PRECISELY IN THIS PART OF THE CODE, PAY ATTENTION TO THE INDENTATION, PYTHON REQUIRES CONSISTENCY IN ORDER FOR THE CODE TO RUN CORRECTLY
    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    '''

    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(inputs.shape[-1], activation = 'softmax')(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    return x + res

In [51]:
# Transformer Decoder function
def transformer_decoder(inputs, enc_outputs, head_size, num_heads, ff_dim, dropout=0.1, l2_reg=0.01):
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(res, enc_outputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = x + res

    # Feed-forward network
    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    '''
    YOU MAY WANT TO ADD OR REMOVE THESE TWO LINES HERE PRECISELY IN THIS PART OF THE CODE, PAY ATTENTION TO THE INDENTATION, PYTHON REQUIRES CONSISTENCY IN ORDER FOR THE CODE TO RUN CORRECTLY
    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    '''

    x = layers.Dense(ff_dim, activation="relu", kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
    x = layers.Dropout(dropout)(x)

    x = layers.Dense(inputs.shape[-1], activation = 'softmax')(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    return x + res


In [52]:
# Build the model
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0, l2_reg=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs

    # Encoder
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout, l2_reg)


    # Decoder
    decoder_input = keras.Input(shape=input_shape)
    dec = decoder_input
    for _ in range(num_transformer_blocks):
        dec = transformer_decoder(dec, x, head_size, num_heads, ff_dim, dropout, l2_reg)

    x = layers.Flatten()(x)
    dec = layers.Flatten()(dec)
    x = layers.Concatenate()([x, dec])

    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model([inputs, decoder_input], outputs)
    return model

In [53]:
# Define the hyperparameters
"""
# THIS IS THE PART YOU SHOULD PLAY AROUND WITH
# YOU MAY REDUCE THESE NUMBERS IF YOUR MACHINE IS OTHERWISE INCAPABLE OF RUNNING THE CODE"""

head_size = 2
num_heads = 2
ff_dim = 128
num_transformer_blocks = 2

mlp_units = [256]
dropout = 0.1
mlp_dropout = 0.1
initial_lr = 0.01
l2_reg=0.01

In [54]:
# Build the model
model = build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout, mlp_dropout)

In [55]:
# Create the optimizer with the initial learning rate
optimizer = Adam(learning_rate=initial_lr)

In [56]:
# Compile the model
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [57]:
# Create the ReduceLROnPlateau callback
lr_callback = ReduceLROnPlateau(factor=0.1, patience=50)

In [58]:
# Record the start time
start_time = time.time()

In [59]:
model.fit([x_train, x_train], y_train,
          batch_size=64,
          epochs=100,
          validation_data=([x_val, x_val], y_val),
          callbacks=[lr_callback])

Epoch 1/100
906/906 [==============================] - 36s 29ms/step - loss: 1.4985 - accuracy: 0.7037 - val_loss: 0.8419 - val_accuracy: 0.7812 - lr: 0.0100
Epoch 2/100
906/906 [==============================] - 26s 29ms/step - loss: 0.7641 - accuracy: 0.7651 - val_loss: 0.6111 - val_accuracy: 0.8025 - lr: 0.0100
Epoch 3/100
906/906 [==============================] - 24s 27ms/step - loss: 0.6787 - accuracy: 0.7789 - val_loss: 0.5177 - val_accuracy: 0.8178 - lr: 0.0100
Epoch 4/100
906/906 [==============================] - 24s 26ms/step - loss: 0.6443 - accuracy: 0.7809 - val_loss: 0.4406 - val_accuracy: 0.8398 - lr: 0.0100
Epoch 5/100
906/906 [==============================] - 24s 27ms/step - loss: 0.6453 - accuracy: 0.7820 - val_loss: 0.4794 - val_accuracy: 0.8287 - lr: 0.0100
Epoch 6/100
906/906 [==============================] - 24s 27ms/step - loss: 0.6430 - accuracy: 0.7838 - val_loss: 0.4452 - val_accuracy: 0.8358 - lr: 0.0100
Epoch 7/100
906/906 [==============================]

In [60]:
# Record the end time
end_time = time.time()

In [61]:
# Calculate the total running time
running_time = end_time - start_time
print("Total Running Time: {:.2f} seconds".format(running_time))

Total Running Time: 2554.27 seconds


In [62]:
# Evaluate the model
loss, accuracy = model.evaluate([x_test, x_test], y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

252/252 [==============================] - 2s 6ms/step - loss: 0.3221 - accuracy: 0.8726
Test loss: 0.3220961391925812, Test accuracy: 0.8726391792297363


In [63]:
# Save the trained model
model.save("my_model_camille.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
